In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("Spark NLP")\
    .master("local[*]")\
    .config("spark.driver.memory","8g") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .config("spark.driver.maxResultSize", "0") \
    .config("spark.kryoserializer.buffer.max", "2000M")\
    .getOrCreate()

[ WARN] Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
surl = 'neo4j://localhost:7692'
spassword = 'password'
turl='neo4j+s://061a1cbc.databases.neo4j.io'
tpassword = 'KuK8K1aTC6_LkCeyBHVvq_JZxA4EC7ooqeNk8LYH8o8'

In [3]:
checkpoint1 = '/home/worker2/Desktop/thesis/dmart_checkpoint/checkpoint1'
checkpoint2 = '/home/worker2/Desktop/thesis/dmart_checkpoint/checkpoint2'
checkpoint3 = '/home/worker2/Desktop/thesis/dmart_checkpoint/checkpoint3'
checkpoint4 = '/home/worker2/Desktop/thesis/dmart_checkpoint/checkpoint4'
checkpoint5 = '/home/worker2/Desktop/thesis/dmart_checkpoint/checkpoint5'
checkpoint6 = '/home/worker2/Desktop/thesis/dmart_checkpoint/checkpoint6'
checkpoint7 = '/home/worker2/Desktop/thesis/dmart_checkpoint/checkpoint7'
checkpoint8 = '/home/worker2/Desktop/thesis/dmart_checkpoint/checkpoint8'
checkpoint9 = '/home/worker2/Desktop/thesis/dmart_checkpoint/checkpoint9'
checkpoint10 = '/home/worker2/Desktop/thesis/dmart_checkpoint/checkpoint10'
checkpoint11 = '/home/worker2/Desktop/thesis/dmart_checkpoint/checkpoint11'

In [4]:
df = spark.readStream.format("org.neo4j.spark.DataSource") \
  .option("url", surl)\
  .option("authentication.type", "basic")\
  .option("authentication.basic.username", "neo4j")\
  .option("authentication.basic.password", spassword)\
  .option("streaming.from", "ALL") \
  .option("streaming.property.name", "id") \
  .option("streaming.query.offset", "MATCH (p:user) RETURN p") \
  .option("query", "match (c:User)-[HAVE_PL]->(pl:ProgrammingLanguage) return c.id as id, c.username as username, c.email as email, pl.name as name") \
  .load()

In [5]:
user = df.dropDuplicates(["id","username",'email']).select(df.id,df.username,df.email)

In [6]:
query1 = user.writeStream.format("org.neo4j.spark.DataSource")\
    .option("save.mode", "Overwrite") \
  .option("url", turl)\
  .option("authentication.type", "basic")\
  .option("authentication.basic.username", "neo4j")\
  .option("authentication.basic.password", tpassword)\
    .option("checkpointLocation", checkpoint1) \
    .option("node.keys", "id")\
    .option("labels",":User")\
    .start()

[ WARN] spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [7]:
pl = df.dropDuplicates(["name"]).select(df.name)
query2 = pl.writeStream.format("org.neo4j.spark.DataSource")\
    .option("save.mode", "Overwrite") \
  .option("url", turl)\
  .option("authentication.type", "basic")\
  .option("authentication.basic.username", "neo4j")\
  .option("authentication.basic.password", tpassword)\
      .option("checkpointLocation", checkpoint2) \
    .option("node.keys", "name")\
    .option("labels",":ProgrammingLanguage")\
    .start()

[ WARN] spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [8]:
user_pl = df.select(df.id, df.name)
query3 = user_pl.writeStream.format("org.neo4j.spark.DataSource")\
    .option("save.mode", "Overwrite") \
  .option("url", turl)\
  .option("authentication.type", "basic")\
  .option("authentication.basic.username", "neo4j")\
  .option("authentication.basic.password", tpassword)\
    .option("checkpointLocation", checkpoint3) \
    .option("relationship", 'HAVE_PL')\
    .option('relationship.save.strategy','keys')\
    .option('relationship.source.labels',':User')\
    .option('relationship.source.node.keys','id:id')\
    .option('relationship.source.save.mode','Match')\
    .option('relationship.target.labels',':ProgrammingLanguage')\
    .option('relationship.target.node.keys','name:name')\
    .option('relationship.target.save.mode','Match')\
    .start()

[ WARN] spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [9]:
df2 = spark.readStream.format("org.neo4j.spark.DataSource") \
  .option("url", surl)\
  .option("authentication.type", "basic")\
  .option("authentication.basic.username", "neo4j")\
  .option("authentication.basic.password", spassword)\
  .option("streaming.from", "ALL") \
  .option("streaming.property.name", "id") \
  .option("streaming.query.offset", "MATCH (p:user) RETURN p") \
  .option("query", "match (c:User)-[HAVE_SKILL]->(t:Tool) return c.id as id, t.name as name") \
  .load()

In [10]:
tool = df2.dropDuplicates(["name"]).select(df2.name)
query4 = tool.writeStream.format("org.neo4j.spark.DataSource")\
    .option("save.mode", "Overwrite") \
  .option("url", turl)\
  .option("authentication.type", "basic")\
  .option("authentication.basic.username", "neo4j")\
  .option("authentication.basic.password", tpassword)\
    .option("checkpointLocation", checkpoint4) \
    .option("node.keys", "name")\
    .option("labels",":Tool")\
    .start()

[ WARN] spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [11]:
user_tool = df2.select(df2.id, df2.name)
query5 = user_tool.writeStream.format("org.neo4j.spark.DataSource")\
    .option("save.mode", "Overwrite") \
  .option("url", turl)\
  .option("authentication.type", "basic")\
  .option("authentication.basic.username", "neo4j")\
  .option("authentication.basic.password", tpassword)\
    .option("checkpointLocation", checkpoint5) \
    .option("relationship", 'HAVE_TOOL')\
    .option('relationship.save.strategy','keys')\
    .option('relationship.source.labels',':User')\
    .option('relationship.source.node.keys','id:id')\
    .option('relationship.source.save.mode','Match')\
    .option('relationship.target.labels',':Tool')\
    .option('relationship.target.node.keys','name:name')\
    .option('relationship.target.save.mode','Match')\
    .start()

[ WARN] spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [12]:
df3 = spark.readStream.format("org.neo4j.spark.DataSource") \
  .option("url", surl)\
  .option("authentication.type", "basic")\
  .option("authentication.basic.username", "neo4j")\
  .option("authentication.basic.password", spassword)\
  .option("streaming.from", "ALL") \
  .option("streaming.property.name", "id") \
  .option("streaming.query.offset", "MATCH (p:user) RETURN p") \
  .option("query", "match (c:User)-[HAVE_SKILL]->(t:Framework) return c.id as id, t.name as name") \
  .load()

In [13]:
fram = df3.dropDuplicates(["name"]).select(df3.name)
query6 = fram.writeStream.format("org.neo4j.spark.DataSource")\
    .option("save.mode", "Overwrite") \
  .option("url", turl)\
  .option("authentication.type", "basic")\
  .option("authentication.basic.username", "neo4j")\
  .option("authentication.basic.password", tpassword)\
    .option("checkpointLocation", checkpoint6) \
    .option("node.keys", "name")\
    .option("labels",":Framework")\
    .start()

[ WARN] spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [14]:
user_fram = df3.select(df3.id, df3.name)
query7 = user_fram.writeStream.format("org.neo4j.spark.DataSource")\
    .option("save.mode", "Overwrite") \
  .option("url", turl)\
  .option("authentication.type", "basic")\
  .option("authentication.basic.username", "neo4j")\
  .option("authentication.basic.password", tpassword)\
  .option("checkpointLocation", checkpoint7) \
    .option("relationship", 'HAVE_FRAM')\
    .option('relationship.save.strategy','keys')\
    .option('relationship.source.labels',':User')\
    .option('relationship.source.node.keys','id:id')\
    .option('relationship.source.save.mode','Match')\
    .option('relationship.target.labels',':Framework')\
    .option('relationship.target.node.keys','name:name')\
    .option('relationship.target.save.mode','Match')\
    .start()

[ WARN] spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [15]:
df4 = spark.readStream.format("org.neo4j.spark.DataSource") \
  .option("url", surl)\
  .option("authentication.type", "basic")\
  .option("authentication.basic.username", "neo4j")\
  .option("authentication.basic.password", spassword)\
  .option("streaming.from", "ALL") \
  .option("streaming.property.name", "id") \
  .option("streaming.query.offset", "MATCH (p:user) RETURN p") \
  .option("query", "match (c:User)-[HAVE_SKILL]->(t:Platform) return c.id as id, t.name as name") \
  .load()

In [16]:
plat = df4.dropDuplicates(["name"]).select(df4.name)
query8 = plat.writeStream.format("org.neo4j.spark.DataSource")\
    .option("save.mode", "Overwrite") \
  .option("url", turl)\
  .option("authentication.type", "basic")\
  .option("authentication.basic.username", "neo4j")\
  .option("authentication.basic.password", tpassword)\
    .option("checkpointLocation", checkpoint8) \
    .option("node.keys", "name")\
    .option("labels",":Platform")\
    .start()

[ WARN] spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [17]:
user_plat = df4.select(df4.id, df4.name)
query9 = user_plat.writeStream.format("org.neo4j.spark.DataSource")\
    .option("save.mode", "Overwrite") \
  .option("url", turl)\
  .option("authentication.type", "basic")\
  .option("authentication.basic.username", "neo4j")\
  .option("authentication.basic.password", tpassword)\
  .option("checkpointLocation", checkpoint9) \
    .option("relationship", 'HAVE_PLAT')\
    .option('relationship.save.strategy','keys')\
    .option('relationship.source.labels',':User')\
    .option('relationship.source.node.keys','id:id')\
    .option('relationship.source.save.mode','Match')\
    .option('relationship.target.labels',':Platform')\
    .option('relationship.target.node.keys','name:name')\
    .option('relationship.target.save.mode','Match')\
    .start()

[ WARN] spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [18]:
df5 = spark.readStream.format("org.neo4j.spark.DataSource") \
  .option("url", surl)\
  .option("authentication.type", "basic")\
  .option("authentication.basic.username", "neo4j")\
  .option("authentication.basic.password", spassword)\
  .option("streaming.from", "ALL") \
  .option("streaming.property.name", "id") \
  .option("streaming.query.offset", "MATCH (p:user) RETURN p") \
  .option("query", "match (c:User)-[HAVE_KNOW]->(t:Knowledge) return c.id as id, t.name as name") \
  .load()

In [19]:
know = df5.dropDuplicates(["name"]).select(df5.name)
query10 = know.writeStream.format("org.neo4j.spark.DataSource")\
    .option("save.mode", "Overwrite") \
  .option("url", turl)\
  .option("authentication.type", "basic")\
  .option("authentication.basic.username", "neo4j")\
  .option("authentication.basic.password", tpassword)\
    .option("checkpointLocation", checkpoint10) \
    .option("node.keys", "name")\
    .option("labels",":Knowledge")\
    .start()

[ WARN] spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [20]:
user_know = df5.select(df5.id, df5.name)
query11 = user_know.writeStream.format("org.neo4j.spark.DataSource")\
    .option("save.mode", "Overwrite") \
  .option("url", turl)\
  .option("authentication.type", "basic")\
  .option("authentication.basic.username", "neo4j")\
  .option("authentication.basic.password", tpassword)\
    .option("checkpointLocation", checkpoint11) \
    .option("relationship", 'HAVE_KNOW')\
    .option('relationship.save.strategy','keys')\
    .option('relationship.source.labels',':User')\
    .option('relationship.source.node.keys','id:id')\
    .option('relationship.source.save.mode','Match')\
    .option('relationship.target.labels',':Knowledge')\
    .option('relationship.target.node.keys','name:name')\
    .option('relationship.target.save.mode','Match')\
    .start()

[ WARN] spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [21]:
query1.awaitTermination()
query2.awaitTermination()
query3.awaitTermination()
query4.awaitTermination()
query5.awaitTermination()
query6.awaitTermination()
query7.awaitTermination()
query8.awaitTermination()
query9.awaitTermination()
query10.awaitTermination()
query11.awaitTermination()

In [ ]:
spark.stop()